## File API Examples

This notebook demonstrates how to work with Evo Files using **direct API calls**, providing granular control over file operations and detailed insight into the underlying service interactions.

### API vs SDK Approach

This notebook uses **direct File API calls** which:
- ✅ Provides full control over API requests and responses
- ✅ Shows detailed API response structures and metadata
- ✅ Allows for advanced customization and fine-tuning
- ✅ Helps understand the underlying service architecture
- ✅ Useful for debugging and advanced use cases

### Want a Simpler, High-Level Interface?

If you prefer a more streamlined experience with less boilerplate code, check out the `sdk-examples.ipynb` notebook in this same directory. The SDK examples show:
- Simplified method calls using `FileAPIClient`
- Automatic handling of complex API interactions
- Direct integration with file uploads and downloads
- Better error handling and validation
- Recommended for most common use cases

In [ ]:
from evo.notebooks import ServiceManagerWidget

cache_location = "./notebook-data"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Prepare Evo SDK parameters

In [ ]:
import os

# Create download directories
download_path = cache_location + "/downloads"
input_path = f"{cache_location}/input"

if not os.path.exists(download_path):
    try:
        os.mkdir(download_path)
    except Exception:
        raise RuntimeError(f"Error: Failed to create {download_path}.")

# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

# Copy the environment details to local variables for easier access.
evo_hub_url = environment.hub_url
org_id = environment.org_id
workspace_id = environment.workspace_id

### Demo 1: Upload a file

In [ ]:
file_path = "data/example.txt"